In [ ]:
# default_exp fastai-sequence

# Action Recognition using Fastai

> Ref. [3D Resnet for NIfTI images](https://gist.github.com/jcreinhold/78943cdeca1c5fca4a5af5d066bd8a8d)

In [ ]:
import cv2
from PIL import Image
from mpl_toolkits.axes_grid1 import axes_grid

import fastai
from fastai.vision import *
from fastai.vision.transform import *
import torchvision.transforms as transforms

from fastai.metrics import error_rate, accuracy

from actions_in_videos.dataset_ucf101 import UCF101
from actions_in_videos.avi import AVI
from actions_in_videos.models_resnet_3d import resnet50

Reload packages where content for package development

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
print(f'fastai version: {fastai.__version__}')
print(f'pytorch version: {torch.__version__}')
print('python version: {}.{}.{}'.format(sys.version_info.major, sys.version_info.minor, sys.version_info.micro))
#print(f'torchvision version: {torchvision.__version__}')

Check GPU

In [ ]:
!nvidia-smi

## Custom DataSets in Fastai
> ref. https://docs.fast.ai/basic_data.html#DataBunch  
https://docs.fast.ai/tutorial.itemlist.html  
https://blog.usejournal.com/finding-data-block-nirvana-a-journey-through-the-fastai-data-block-api-c38210537fe4  

In [ ]:
SEQ_LEN = 16
kinetic_mean = np.asarray([0.433, 0.4045, 0.3776],np.float32)
kinetic_std = np.asarray([0.1519876, 0.14855877, 0.156976],np.float32)

ucf_mean = np.array([0.485, 0.456, 0.406])
ucf_std = np.array([0.229, 0.224, 0.225])

In [ ]:
def tensorsequence2np(seq:Tensor)->np.ndarray:
    "Convert from torch style `sequence` to numpy/matplotlib style."
    # reshape to have the seq as a single bigger image.
    seq = seq.reshape(1,3,seq.shape[0]*seq.shape[2],-1)     
    res = seq.cpu().permute(0,2,3,1).numpy().squeeze()  
    return res[...,0] if res.shape[2]==1 else res

In [ ]:
def seq2tensor(seq:NPArray, dtype:np.dtype)->TensorImage:
    """
    Convert sequence array to torch style image tensor.

    Can be used to convert the output of getVideoSequence into a tensor.
    For example:
    x = getVideoSequence(fn, num_of_frames=self._seq_len)
    f = seq2tensor(x, np.float32)
    f.shape
    torch.Size([16, 3, 240, 320])
    
    Returns:
    - torch tensor of size [seq_len, channel, height, width]
    
    """ 
    try:
        a = np.asarray(seq)
        if a.ndim==3 : a = np.expand_dims(a,3)
        a = np.transpose(a, (0, 3, 1, 2))
    except Exception as e:
        print(e)
#         # set_trace()
#         import pickle
#         f=open('./dump1','wb+'); pickle.dump(a,f); f.close()
        a = np.array([])
    return torch.from_numpy(a.astype(dtype, copy=False) )

In [ ]:
def getVideoSequence(filename, num_of_frames=16):
    """
    Extract a sequence of frames from an avi file.
    Returns: list of frames.
        for ex.  the output of a sequence of sixteen frames in color would be: seq_len, (channel, height, width)
                                                                                    16, (240, 320, 3)
    """
    ### load file from AVI
    cap = cv2.VideoCapture(str(filename))
    if not cap.isOpened(): 
        print(f"could not open {filename}") 
        return [] 
    nFrames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps     = cap.get(cv2.CAP_PROP_FPS)
    frame_index = np.random.randint(nFrames - num_of_frames)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)    
    try:
        video = []        
        f_count = 0
        while cap.isOpened() and f_count < num_of_frames:
            frameId = cap.get(cv2.CAP_PROP_POS_FRAMES) # current frame number
            ret, frame = cap.read() 
            video.append(frame)
            f_count += 1
        cap.release() 
    except:
        print("\nException: " + filename)
        video = [] 
        cap.release() 
        set_trace()
        
    # output is with values [0, 255]
    return video

In [ ]:
class Sequence(ItemBase): 
    """ Support applying transforms to sequence data (tensor of images) in `px`.        
        As all ItemBase classes, it is not usable until you have put it inside your custom ItemList though.        
        """
    def __init__(self, px:Tensor):
        self._obj = px # this is the original data.
        self._px  = px # this is the property that gets changed.
        self._seq_len = self._px.shape[0]        
        
    def clone(self):
        "Mimic the behavior of torch.clone for `Sequence` objects."
        return self.__class__(self._px.clone())
     
    @property
    def shape(self)->Tuple[int,int,int,int]: return self._px.shape
    @property
    def size(self)->Tuple[int,int,int]: return self.shape[-2:]
    @property
    def device(self)->torch.device: return self._px.device
    
    def __len__(self):
        return len(self._px[0])
    
    def __repr__(self): return f'{self.__class__.__name__} {tuple(self.shape)}'
    def _repr_png_(self):  return self._repr_image_format('png')
    
    def _repr_image_format(self, format_str):        
        rows = int(np.ceil(math.sqrt(self._seq_len)))
        axs = subplots(rows, rows)      
        for x, ax in zip(self._px, axs.flatten()): Image(x).show(ax=ax)
        for ax in axs.flatten()[len(self._px):]: ax.axis('off')
        plt.tight_layout()            
              
    def apply_tfms_per_image(self, tfms:TfmList, do_resolve:bool=True, xtra:Optional[Dict[Callable,dict]]=None,
                   size:Optional[Union[int,TensorImageSize]]=None, resize_method:ResizeMethod=None,
                   mult:int=None, padding_mode:str='reflection', mode:str='bilinear', remove_out:bool=True)->TensorImage:
        "Apply all `tfms` to each `Sequence Image`, if `do_resolve` picks value for random args."           
        print('apply_tfms')
        px_list = []
        for i,x in enumerate(self._px):
            x = Image(x)
            x = x.apply_tfms(tfms, do_resolve, xtra, size, resize_method, mult, padding_mode, mode, remove_out)
            px_list.append(x.data.numpy())        
        res = np.asarray(px_list, np.float32)
        self._px = torch.from_numpy(res.astype(np.float32, copy=False) )
        return self
    
    def apply_tfms(self, tfms:TfmList, do_resolve:bool=True, xtra:Optional[Dict[Callable,dict]]=None,
                   size:Optional[Union[int,TensorImageSize]]=None, resize_method:ResizeMethod=None,
                   mult:int=None, padding_mode:str='reflection', mode:str='bilinear', remove_out:bool=True)->TensorImage:
        "Apply all `tfms` to a `Sequence`"    
        #print('len of tfms:',len(tfms))
        seq = np.asarray(self._px, np.float32)        
        if len(tfms) > 1:
            # training
            seq = self.transform_sequence(seq, training=True, stats=(kinetic_mean, kinetic_std), size=(224, 224), p_flip=0.5, p_crop=0.5, crop_pc=0.7)
        else:
            seq = self.transform_sequence(seq, training=False, stats=(kinetic_mean, kinetic_std), size=(224, 224), p_flip=0.5, p_crop=0.5, crop_pc=0.7)
        #set_trace()
        self._px = seq #torch.from_numpy(seq.astype(np.float32, copy=False) )
        return self

 
    def transform_sequence(self, clip, training, stats, size=(224, 224), p_flip=0.5, p_crop=0.5, crop_pc=0.7, brightness=30, verbose=False):     
        clip = clip.transpose(0,2,3,1) # seq_len, h, w, c
        curr_h, curr_w, channels = clip[0].shape  # (240, 320, 3) -> curr_w = 320, curr_h = 240
        height, width = size

        mean, std = stats

        try:            
            if training:
                ## RANDOM CROP - crop 70-100% of original size
                ## don't maintain aspect ratio            
                resize_factor_w = (1-crop_pc)*np.random.rand()+crop_pc
                resize_factor_h = (1-crop_pc)*np.random.rand()+crop_pc
                w1 = int(curr_w*resize_factor_w)
                h1 = int(curr_h*resize_factor_h)
                w = np.random.randint(curr_w-w1)
                h = np.random.randint(curr_h-h1)
                coin_crop = np.random.uniform()

                coin_flip = np.random.uniform()

                ## Brightness +/- 15
                random_add = np.random.randint(brightness+1) - brightness/2.0
                random_add /=255

                data = []
                for i, frame in enumerate(clip):
                    if verbose: print(f'frame n. {i}')
                    if coin_crop < p_crop:
                        if verbose: print("Add random crop")
                        frame = frame[h:(h+h1),w:(w+w1),:]
                    if coin_flip < p_flip:
                        if verbose: print("Add random flip")                    
                        frame = cv2.flip(frame,1)

                    if verbose: print(f'Resizing to ({width,height})')
                    frame = cv2.resize(frame,(width,height))
                    frame = frame.astype(np.float32)

                    if verbose: print(f"Add random brightness value {random_add}")                    
                    frame += random_add
                    frame[frame>1] = 1.0
                    frame[frame<0] = 0.0

                    # Normalise
                    if verbose: 
                        print(f"Normalise with mean {mean} and std {std}")                                
                        print(f"   original: np.mean(frame): {np.mean(frame,axis=(0, 1))}")
                        print(f"   original: np.std(frame): {np.std(frame,axis=(0, 1))}")                
                    frame = (frame - mean)/std
                    if verbose:
                        print(f"   normalised: np.mean(frame): {np.mean(frame,axis=(0, 1))}")
                        print(f"   normalised: np.std(frame): {np.std(frame,axis=(0, 1))}")                
                    data.append(frame)
                data = np.asarray(data)
            else:
                # don't augment
                data = []
                for frame in clip:
                    frame = cv2.resize(frame,(width,height))
                    frame = frame.astype(np.float32)
                    frame = (frame - mean)/std
                    data.append(frame)
                data = np.asarray(data)

        except:
            print("\nException: transform_sequence failed. ")
            data = np.array([])       
            set_trace()
        # data.shape is: seq_len, h, w, c - this is also the avi.getSequence output                        
        # we transpose it to the tensor shape
        data = seq2tensor(data, np.float32) #.transpose(0,3,1,2) # seq_len, c, h, w
        
        return data
    
    
    @property
    def data(self)->TensorImage:
        "Return this sequence pixels as a tensor."
        return self._px
    
    def show(self, ax:plt.Axes=None, figsize:tuple=(3,3), title:Optional[str]=None, hide_axis:bool=True,
              cmap:str=None, y:Any=None, **kwargs):
        "Show image on `ax` with `title`, using `cmap` if single-channel, overlaid with optional `y`"
        cmap = ifnone(cmap, defaults.cmap)   
        if self._seq_len > 1:
            image = self._px[0]    
        else: image = self._px
        ax = show_image(image, ax=ax, hide_axis=hide_axis, cmap=cmap, figsize=figsize)
        if y is not None: y.show(ax=ax, **kwargs)
        if title is not None: ax.set_title(title)               

Try and use `SequenceDataBunch` to normalise the data. Note that this requires setting the `_bunch` attribute to `SequenceDataBunch` in class `SequenceList(ItemList)`

In [ ]:
def channel_view_seq(x:Tensor)->Tensor:
    "Make channel the first axis of `x` of a sequence and flatten remaining axes"
    return x.transpose(0,2).contiguous().view(x.shape[2],-1)


class SequenceDataBunch(DataBunch):
    def batch_stats(self, funcs:Collection[Callable]=None, ds_type:DatasetType=DatasetType.Train)->Tensor:
        "Grab a batch of data and call reduction function `func` per channel"
#        set_trace()
        funcs = ifnone(funcs, [torch.mean,torch.std])
        x = self.one_batch(ds_type=ds_type, denorm=False)[0].cpu()
        return [func(channel_view_seq(x), 1) for func in funcs]

    def normalize(self, stats:Collection[Tensor]=None, do_x:bool=True, do_y:bool=False)->None:
        "Add normalize transform using `stats` (defaults to `DataBunch.batch_stats`)"
        if getattr(self,'norm',False): raise Exception('Can not call normalize twice')
        if stats is None: self.stats = self.batch_stats()
        else:             self.stats = stats
        self.norm,self.denorm = normalize_funcs(*self.stats, do_x=do_x, do_y=do_y)
        self.add_tfm(self.norm)
        print(self.add_tfm)
        return self
    

In [ ]:
class SequenceList(ItemList): 
    """
       Subclass ImageList to use our own image opening function
       This is the main class that allows you to group your inputs or your targets in the data block API. 
       You can then use any of the splitting or labelling methods before creating a DataBunch."""
    
    # _processor
    # _bunch = SequenceDataBunch
    _label_cls=None  # If set to one, fastai selects one class that should fit.
    
    _seq_len=SEQ_LEN
    def __init__(self, items, **kwargs):
        super().__init__(items, **kwargs)          
      
    def open(self, fn:PathOrStr, seq_len=16)->Image:
        #avi = AVI(fn, verbose=False, img_type=np.float32)
        #x = avi.getRandomSequence(seq_len=seq_len, sample_interval=1)
        got_video=False
        while(got_video==False):
            x = getVideoSequence(fn, num_of_frames=self._seq_len)
            ## This ndarray image has to be converted to tensor before passing on as fastai Image, we can use pil2tensor                               
            x = seq2tensor(x, np.float32)
            if len(x.shape) == 4:
                got_video = True
                break
            print(f'Error while reading video: {fn}')        
        x.div_(255)  # values are now [0,1]
        return Sequence(x)            
    
    def get(self, i):
        "Defines how to construct an ItemBase from the data in the ItemList.items array"
        fn = super().get(i)
        res = self.open(fn, seq_len=self._seq_len)
        return res
    
    def reconstruct(self, t:Tensor): 
        # this is to remove normalisation if was applied before:        
#         mean = np.asarray([0.433, 0.4045, 0.3776],np.float32)
#         std = np.asarray([0.1519876, 0.14855877, 0.156976],np.float32)
#         seq = np.zeros(t.shape)
#         for i, img in enumerate(t):
#             for j, (f, m, s) in enumerate(zip(img, mean, std)):  # loop through the channels:  
#                 seq[i,j,:,:]=f*s+m # seq = std*t.numpy() + mean        
#         return Sequence(torch.from_numpy(seq.astype(np.float32, copy=False) ))
        ######################
        return Sequence(t.float().clamp(min=0,max=1))
            
    @classmethod
    def from_folder(cls, path, **kwargs):
        res = super().from_folder(path, **kwargs)
        res.path = path
        return res
    
    def split_by_files(self, names):
        return self.split_by_valid_func(lambda o: any(o.name in vn for vn in names))
    
    def split_by_fname_file(self, fname, path=None, **kwargs):        
        "Split the data by using the names in `fname` for the validation set. `path` will override `self.path`."        
        path = Path(ifnone(path, self.path))
        relative_valid_names = loadtxt_str(path/fname)
        valid_names = [str(Path(self.path).joinpath(Path(fn))) for fn in relative_valid_names]        
        return self.split_by_files(valid_names)                    
    
    def show_xys(self, xs, ys, imgsize:int=4, figsize:Optional[Tuple[int,int]]=None, **kwargs):
        "Show the `xs` (inputs) and `ys` (targets) on a figure of `figsize`."
        rows = int(np.ceil(math.sqrt(len(xs))))
        axs = subplots(rows, rows, imgsize=imgsize, figsize=figsize)
        for x,y,ax in zip(xs, ys, axs.flatten()):             
            x.show(ax=ax, y=y, **kwargs)
        for ax in axs.flatten()[len(xs):]: ax.axis('off')
        plt.tight_layout()        

    def show_xyzs(self, xs, ys, zs, imgsize:int=4, figsize:Optional[Tuple[int,int]]=None, **kwargs):
        "Show `xs` (inputs), `ys` (targets) and `zs` (predictions) on a figure of `figsize`."
        if self._square_show_res:
            title = 'Ground truth\nPredictions'
            rows = int(np.ceil(math.sqrt(len(xs))))
            axs = subplots(rows, rows, imgsize=imgsize, figsize=figsize, title=title, weight='bold', size=12)
            for x,y,z,ax in zip(xs,ys,zs,axs.flatten()): x.show(ax=ax, title=f'{str(y)}\n{str(z)}', **kwargs)
            for ax in axs.flatten()[len(xs):]: ax.axis('off')
        else:
            title = 'Ground truth/Predictions'
            axs = subplots(len(xs), 2, imgsize=imgsize, figsize=figsize, title=title, weight='bold', size=14)
            for i,(x,y,z) in enumerate(zip(xs,ys,zs)):
                x.show(ax=axs[i,0], y=y, **kwargs)
                x.show(ax=axs[i,1], y=z, **kwargs)

In [ ]:
# data = (SequenceList.from_folder('../data/UCF101/UCF-101/'))

In [ ]:
# data[101]

In [ ]:
def sequence_collate(batch):
    try:
        len_batch = len(batch) # original batch length
        batch = list(filter (lambda x:x is not None, batch)) # filter out all the Nones
        batch = list(filter (lambda x: len(x[0])>0, batch))  # filter out all the empty ones.
        
        if len_batch > len(batch): 
            # if there are samples missing just use existing members, doesn't work if you reject every sample in a batch
            diff = len_batch - len(batch)
            print(f'sequence_collate - diff is {diff}')
            
        batch = torch.utils.data.dataloader.default_collate(to_data(batch))        
    except Exception as e:            
        print(e)
        set_trace()
    return batch

In [ ]:
def get_y_func(o):
    if str(o.parents[0]).split('/')[-1] == 'models':
        print(o)
        print(str(o.parents[0]).split('/')[-1]) 
    return str(o.parents[0]).split('/')[-1]

get_y = lambda o:str(o.parents[0]).split('/')[-1]

In [ ]:
tfms = get_transforms(do_flip=False, flip_vert=False, max_rotate=0.0, p_affine=0.5, p_lighting=0.5)

In [ ]:
len(tfms[1])

In [ ]:
data_path = '../data/UCF101/UCF-101/'

In [ ]:
data = (SequenceList.from_folder(data_path) # Where to find the data? -> in path and its subfolders
        .split_by_fname_file(fname='testlist01.txt', path='../data/UCF101/annotations/ucfTrainTestlist/') 
        .label_from_func(get_y_func)         # How to label
        .transform(tfms, size=224)           # Data augmentation? -> use tfms with a size of 224
        .databunch(bs=64, collate_fn=sequence_collate))    

In [ ]:
data.show_batch()

In [ ]:
data.show_batch(ds_type=DatasetType.Valid)

If we use `SequenceDataBunch` that we can calcuate some statistics. 

Unnormalised statistics:

In [ ]:
data.batch_stats()

`SequenceDataBunch` also makes available a normalise method:

In [ ]:
data = data.normalize((kinetic_mean, kinetic_std))

In [ ]:
data.batch_stats()

In [ ]:
data.show_batch()

In [ ]:
# data_path = '../data/UCF101/UCF-5'

In [ ]:
# data = (SequenceList.from_folder(data_path) # Where to find the data? -> in path and its subfolders
#         .split_by_fname_file(fname='ucf-5-testlist01.txt', path='../data/UCF101/annotations/ucfTrainTestlist/')             #How to split in train/valid? -> use the folders
#         .label_from_func(get_y_func)         # How to label
#         .transform(tfms, size=224)           # Data augmentation? -> use tfms with a size of 224
#         .databunch(collate_fn=sequence_collate))    

In [ ]:
len(data.train_ds.y.classes)

101

In [ ]:
model =  resnet50(sample_size=224, sample_duration=SEQ_LEN)
pretrained = torch.load('./model-pretrained/resnet-50-kinetics.pth', map_location=torch.device('cpu'))

keys = [k for k,v in pretrained['state_dict'].items()]
pretrained_state_dict = {k[7:]: v.cpu() for k, v in pretrained['state_dict'].items()}
model.load_state_dict(pretrained_state_dict)
model.fc = nn.Linear(model.fc.weight.shape[1], len(data.train_ds.y.classes))

function resnet50


In [ ]:
# model

In [ ]:
for param in model.parameters():
    param.requires_grad_(False)

# for param in model.conv1.parameters():
#     param.requires_grad_(True)
# for param in model.bn1.parameters():
#     param.requires_grad_(True)
# for param in model.layer1.parameters():
#     param.requires_grad_(True)
# for param in model.layer2.parameters():
#     param.requires_grad_(True)
# for param in model.layer3.parameters():
#     param.requires_grad_(True)
for param in model.layer4[0].parameters():
    param.requires_grad_(True)
for param in model.fc.parameters():
    param.requires_grad_(True)

In [ ]:
#model

In [ ]:
loss = nn.CrossEntropyLoss()
learner = Learner(data, model, loss_func=loss, metrics=[accuracy], path='./', callback_fns=ShowGraph)

In [ ]:
learner.summary()

In [ ]:
learner.lr_find()
learner.recorder.plot()

In [ ]:
learner.fit_one_cycle(5,3e-3)

In [ ]:
preds,y,losses = learn.get_preds(with_loss=True)
interp = ClassificationInterpretation(learn, preds, y, losses)

In [ ]:
interp.plot_confusion_matrix(figsize=(20,20), dpi=60)
interp.plot_top_losses(9, figsize=(20,20), heatmap=True)

In [ ]:
tfm = rand_zoom(scale=(1.,1.5))

rand_resize_crop(size:int, max_scale:float=2.0, ratios:Point=(0.75, 1.33))
brightness(x, change:uniform)
contrast(x, scale:log_uniform) 
crop(x, size, row_pct:uniform=0.5, col_pct:uniform=0.5)
crop_pad(x, size, padding_mode='reflection', row_pct:uniform=0.5, col_pct:uniform=0.5)
perspective_warp(c, magnitude:partial(uniform, size=8)=0, invert=False)


RESIZE: data = src.transform(tfms=tfms, size=size, resize_method=ResizeMethod.SQUISH).databunch(bs=bs, num_workers=4).normalize()
    squish(scale:uniform=1.0, row_pct:uniform=0.5, col_pct:uniform=0.5) 

In [ ]:
learner.sched.plot_lr()

In [ ]:
from nbdev.export import *
notebook2script()